Enter OpenAI API Key Path

In [1]:
api_key_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/openai_key.txt"

01 - Generate JSON from Post-Inclusion/Exclusion CSV
- Enter path to CSV generated from Notebook 04 (inclusion/exclusion CSV)
- Enter path to JSON which has the articles with labeled sections

In [2]:
csv_path = "/Users/cu135/Dropbox (Partners HealthCare)/studies/CLOCC/search_results/case_extractions/inclusion_exclusion_results/processed_results.csv"
json_file_path = "/Users/cu135/Dropbox (Partners HealthCare)/studies/CLOCC/search_results/json/_case_labeled_sections.json"

In [3]:
from calvin_utils.gpt_sys_review.json_utils import FilterPapers

# Initialize and run the FilterPapers class
filter_papers = FilterPapers(csv_path=csv_path, json_path=json_file_path)
filtered_json_path = filter_papers.run()

/Users/cu135/.local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


02 - Prepare Data Extraction Questions
- Dev Note: This is a repeat class and should be inherited from a .py

In [4]:
from calvin_utils.gpt_sys_review.gpt_utils import QuestionTemplate

question_template = QuestionTemplate()
question_template.data_extraction_questions()

Here are example data extraction questions:
{
    "Does the patient(s) represent(s) the whole experience of the investigator (center) or is the selection method unclear to the extent that other patients with similar presentation may not have been reported? (Good/Bad/Unclear)": "representative_case_quality",
    "Was patient\u2019s causal exposure clearly described? (Good/Bad/Unclear)": "causality_quality",
    "Were diagnostic tests or assessment methods and the results clearly described (amnesia tests)? (Good/Bad/Unclear)": "phenotyping_quality",
    "Were other alternative causes that may explain the observation (amnesia) ruled out? (Good/Bad/Unclear)": "workup_quality",
    "Were patient\u2019s demographics, medical history, comobidities clearly described? (Good/Bad/Unclear)": "clinical_covariates_quality",
    "Were patient\u2019s symptoms, interventions, and clinical outcomes clearly presented as a timeline? (Good/Bad/Unclear)": "history_quality",
    "Was the lesion image taken a

Copy the Template Dict from Above and Fill it Out As per the Example

In [5]:
question = {
    "Prioritizing implicit and explicit information, do you think this was reasonably free from selection bias? (Yes/No)": "selection_bias",
    "Prioritizing implicit and explicit information, do you think the brain lesion onset and timing was well documented, such as in the history of presenting illness? (Yes/No)": "exposure",
    "Prioritizing implicit and explicit information, do you think the brain lesion was attributable to the symptoms? (Yes/No)": "outcome",
    "Prioritizing implicit and explicit information, do you think this case was adequately worked up to consolidate diagnosis of the brain lesion? (Yes/No)": "attribution_error",
    "Prioritizing implicit and explicit information, do you think this case had reasonable description of past medical history, family history, and medications? (Yes/No)": "pre_event_assessment",
    "Prioritizing implicit and explicit information, do you think this case had reasonable description of the patient post-neurological lesion? (Yes/No)": "post_event_assessment",
    "Prioritizing implicit and explicit information, do you think this case has reasonably high quality imaging of the brain lesion? (Yes/No)": "image_quality",
    "Prioritizing implicit and explicit information, do you think this case was described to the degree that another doctor could reasonably make their own conclusions? (Yes/No)": "replicability",
    "Prioritizing implicit and explicit information, do you think this case is suitable for lesion network mapping? (Yes/No)": "overall_quality",
    "Please list all neurological/psychiatric/cognitive symptoms experienced by this patient.": "symptom_profile"
    }

03 - Extract Data with GPT

Define the segmented labels you want to consider. 

- Article type 'case' will has sections 'case_report' and 'other'
- Article type 'research' has sections "Abstract", "Introduction", "Methods", "Results", "Discussion", "Conclusion", "References"


In [6]:
# Define the keys you want to consider (exclude 'References')
keys_to_consider = ["case_report"]  # Add or remove keys as per your requirement

# Define the type of article and questions
article_type = "case"

In [7]:
test_mode=False

In [8]:
from calvin_utils.gpt_sys_review.gpt_utils import OpenAIChatEvaluator
evaluator = OpenAIChatEvaluator(api_key_path=api_key_path,
                                json_file_path=filtered_json_path, 
                                keys_to_consider=keys_to_consider,
                                question_type=article_type,
                                question=question,
                                test_mode=test_mode,
                                model_choice="gpt4",
                                debug=False)
answers = evaluator.evaluate_all_files()
evaluated_json_path = evaluator.save_to_json(answers)

  0%|          | 0/37 [00:00<?, ?it/s]

100%|██████████| 37/37 [08:19<00:00, 13.51s/it]

Saved to: /Users/cu135/Dropbox (Partners HealthCare)/studies/CLOCC/search_results/case_extractions/inclusion_exclusion_results/inclusion_exclusion_json/../case_extractions/case_evaluations.json


04 - Convert results to a CSV
- Set answers_binary to False if the questions you asked do not have binary answers. 
   - We will extract the raw data, like specific result values, for you to review.
- Set asnwers_binary to True if the questions you asked do have binary answers. 
   - By default, we will set positive answers to 1, and negative answers to 0.

In [17]:
answers_binary=False

In [18]:
from calvin_utils.gpt_sys_review.json_utils import CustomSummarizer
custom_summarizer = CustomSummarizer(json_path=evaluated_json_path, answers_binary=answers_binary)
df, raw_path, automated_path = custom_summarizer.run_custom()

Your CSV files of filtered manuscripts have been saved to this directory: 
 /Users/cu135/Dropbox (Partners HealthCare)/studies/CLOCC/search_results/case_extractions/inclusion_exclusion_results/inclusion_exclusion_json/../case_extractions/data_extraction


05 - Postprocess

- If you have been using a master_list, you can update it with the results from the generated CSVs.

In [19]:
master_list_path = '/Users/cu135/Dropbox (Partners HealthCare)/studies/CLOCC/master_list.csv'

In [20]:
from calvin_utils.gpt_sys_review.txt_utils import PostProcessing
PostProcessing.add_raw_results_to_master_list(master_list_path=master_list_path, raw_results_path=raw_path)

Updating master list: 37it [00:00, 577.94it/s]


,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,...,"Prioritizing implicit and explicit information, do you think this was reasonably free from selection bias? (Yes/No)","Prioritizing implicit and explicit information, do you think the brain lesion onset and timing was well documented, such as in the history of presenting illness? (Yes/No)","Prioritizing implicit and explicit information, do you think the brain lesion was attributable to the symptoms? (Yes/No)","Prioritizing implicit and explicit information, do you think this case was adequately worked up to consolidate diagnosis of the brain lesion? (Yes/No)","Prioritizing implicit and explicit information, do you think this case had reasonable description of past medical history, family history, and medications? (Yes/No)","Prioritizing implicit and explicit information, do you think this case had reasonable description of the patient post-neurological lesion? (Yes/No)","Prioritizing implicit and explicit information, do you think this case has reasonably high quality imaging of the brain lesion? (Yes/No)","Prioritizing implicit and explicit information, do you think this case was described to the degree that another doctor could reasonably make their own conclusions? (Yes/No)","Prioritizing implicit and explicit information, do you think this case is suitable for lesion network mapping? (Yes/No)",Please list all neurological/psychiatric/cognitive symptoms experienced by this patient.
0,28165876,Cytotoxic Lesions of the Corpus Callosum That ...,"Starkey J, Kobayashi N, Numaguchi Y, Moritani T.",Radiographics. 2017 Mar-Apr;37(2):562-576. doi...,Starkey J,Radiographics,2017,2/7/17,NaN,NaN,...,"(""The case report doesn't provide enough infor...","('No', 'No')","('Yes', 'Yes')","('Yes', 'Yes')","('No', 'No')","('No', 'Yes')",('The case report does not provide specific im...,"('Yes', 'Yes')","('No', 'No')",('The case report does not provide specific in...
1,31588684,Reversible lesion in the splenium of the corpu...,Tetsuka S.,Brain Behav. 2019 Nov;9(11):e01440. doi: 10.10...,Tetsuka S,Brain Behav,2019,10/8/19,PMC6851813,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32030930,[Cytotoxic lesion of the corpus callosum (CLOC...,"Hoëffelin H, Marchetti P, Khamis J, Reginster P.",Rev Med Liege. 2020 Feb;75(2):78-82.,Hoëffelin H,Rev Med Liege,2020,2/8/20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38147170,Cytotoxic lesions of the corpus callosum: a sy...,"Moors S, Nakhostin D, Ilchenko D, Kulcsar Z, S...",Eur Radiol. 2023 Dec 26. doi: 10.1007/s00330-0...,Moors S,Eur Radiol,2023,12/26/23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,35220561,A cytotoxic lesion of the corpus callosum (CLO...,"Özdemir HN, Çelebisoy N, Gökçay F, Gökçay A.",Acta Neurol Belg. 2023 Apr;123(2):619-621. doi...,Özdemir HN,Acta Neurol Belg,2023,2/27/22,NaN,NaN,...,"('Yes',)","('Yes',)","('Yes',)","('Yes',)","('No',)","('Yes',)","('Yes',)","('Yes',)","(""The case report doesn't provide explicit inf...",('The patient presented with left-sided hemipa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,23054449,Engaging Chicago hospitals in the baby-friendl...,"Schoenfelder SL, Wych S, Willows CA, Harringto...",Matern Child Health J. 2013 Nov;17(9):1712-7. ...,Schoenfelder SL,Matern Child Health J,2013,10/12/12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,28360602,A Case of ADEM Mimicking Cerebral Adrenoleukod...,"Beyazal M, Ünal Ö, Yilmaz S, Bora A.",Noro Psikiyatr Ars. 2014 Mar;51(1):86-89. doi:...,Beyazal M,Noro Psikiyatr Ars,2014,4/1/17,PMC5370266,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,26306930,Reversible restricted-diffusion lesion represe...,"Al Brashdi YH, Albayram MS.",Neuroradiol J. 2015 Aug;28(4):409-12. doi: 10....,Al Brashdi YH,Neuroradiol J,2015,8/27/15,PMC4757308,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,17602334,[Cerebral vasoreactivity in high-altitude ce

Your articles have been completely evaluated. 

Please check the CSVs in the directory noted above and use the path to the one you would like to use for your further analysis.
- Enjoy. If this has been helpful, please consider adding Calvin Howard as a collaborator. 
- e: choward12@bwh.harvard.edu